In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('immobiliers.csv')

In [ ]:
df.head()

In [ ]:
# on ne s'interesse que sur les biens à vendre
# execute right after reading the csv, marra wahda barka
df = df[df['Type de transaction'] == 'À Vendre']
df = df.drop(columns=['Type de transaction'])
df.reset_index(drop=True, inplace=True)
print(df.info())

In [ ]:
condition1 = df[(df['Configuration des pièces'].notna()) & (df['Chambres']=='False')]
# print(condition1.shape)
# 11 row où nb chambres n'est affiché que dans l'annonce

# Extraire le nombre de pièces de 'Configuration des pièces' et l'attribuer à 'Chambres'
for idx, row in condition1.iterrows():
    config_pieces = row['Configuration des pièces']
    match = re.search(r'S\+(\d+)', config_pieces)
    if match:
        df.at[idx, 'Chambres'] = int(match.group(1))

df = df.drop(columns=['Configuration des pièces']) # on n'en a plus besoin
df.tail()

In [ ]:
# Remplacer toutes les valeurs 'False' dans les colonnes 'Superficie' et 'Chambres' par NaN
df['Superficie'] = df['Superficie'].replace(['False'], np.nan)
df['Chambres'] = df['Chambres'].replace(['False'], np.nan)

# Remplacer toutes les valeurs 'False' dans la colonne "Salles de bains" par la valeur par défaut 1
df['Salles de bains'].replace('False', 1, inplace=True)

df['Prix'] = pd.to_numeric(df['Prix'], errors='coerce') # convertir en numérique

In [ ]:
df.tail()

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
# On s'interesse dans cette partie aux valeurs manquantes dans la colonne type de biens
print("Value counts for 'Type de bien':\n", df['Type de bien'].value_counts())
print("Missing values in 'Type de bien' column: ", df['Type de bien'].isnull().sum())

In [9]:
keywords = {
    'appartement': 'Appartement',
    'villa': 'Villa',
    'maison': 'Maison'
}
# chercher dans les annonces les keywords qui nous interessent
for keyword, replacement in keywords.items():
    condition = df['Annonce'].str.contains(keyword, case=False, na=False)
    df.loc[condition & df['Type de bien'].isna(), 'Type de bien'] = replacement

In [10]:
# suite aux observations, les données non labelisées correspondent à des appartements
df.loc[df['Type de bien'].isna() & df['Chambres'].notna(), 'Type de bien'] = 'Appartement'

In [ ]:
# verification
print("Value counts for 'Type de bien':\n", df['Type de bien'].value_counts())
print("Missing values in 'Type de bien' column: ", df['Type de bien'].isnull().sum())

In [12]:
# Apres la visualtion des nvlles données, on a remarqué que les données 
# manquantes restantes dans la colonne 'Type de bien' sont des
# biens commerciaux dont on n'aura pas besoin reellement

# Garder uniquement les lignes où 'Type de bien' est 'appartement', 'maison', ou 'villa'
df = df[df['Type de bien'].isin(['Appartement', 'Maison', 'Villa'])]

df = df.drop(columns=['Annonce']) # on n'en a plus besoin

In [ ]:
df.head()

In [ ]:
q99 = df['Prix'].quantile(0.99)
filtered_df = df[df['Prix'] <= q99]

sns.boxplot(x='Type de bien', y='Prix', data=filtered_df)
plt.title("Répartition des prix par type de bien")
plt.xlabel("Type de bien")
plt.ylabel("Prix")
plt.show()

In [ ]:
# Imputation basée sur la médiane des prix groupée par Localisation, Nombre de chambres, et Type de bien
df['Prix'] = df.groupby(['Localisation', 'Chambres', 'Type de bien'])['Prix'].transform(
    lambda x: x.fillna(x.median())
)
df['Prix'] = df.groupby(['Localisation', 'Chambres'])['Prix'].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()

In [27]:
# supprimer les annonces sans détails
df.dropna(subset=['Prix', 'Chambres', 'Superficie'], inplace=True)

In [28]:
# garder que l'intervalle logique du nombre de chambres
df['Chambres'] = pd.to_numeric(df['Chambres'], errors='coerce')
df = df[(df['Chambres'] >= 0) & (df['Chambres'] <= 14)]
# print(df['Chambres'].unique())

In [29]:
df.to_csv('cleaned_immobiliers.csv', index=False, encoding='utf-8')